In [1]:
import os
import pandas as pd
print(os.getcwd())

try:
    os.chdir("../../workshop_02")

except FileNotFoundError:
    print("Directory not found or already exists.")
    
print(os.getcwd())

/home/edwards/Documentos/university/etl/workshop_02/notebooks
/home/edwards/Documentos/university/etl/workshop_02


In [ ]:
import pandas as pd
import os
spotify_path = os.path.join('data', 'processed', 'spotify_dataset_cleaned.csv')
grammys_path = os.path.join('data', 'grammys.csv')
api_artist_path = os.path.join('data', 'api_artist.csv')
output_path = os.path.join('data', 'merge_dataset.csv')
os.makedirs('data', exist_ok=True)
spotify_df = pd.read_csv(spotify_path)
grammys_df = pd.read_csv(grammys_path)
api_artist_df = pd.read_csv(api_artist_path)

merged_df = pd.merge(
    spotify_df,
    api_artist_df[['track_id', 'artist_id', 'artist_followers', 'artist_popularity']],
    on='track_id',
    how='left'
)
merged_df['track_name_norm'] = merged_df['track_name'].str.lower().str.strip()
merged_df['artists_norm'] = merged_df['artists'].str.lower().str.strip()
grammys_df['nominee_norm'] = grammys_df['nominee'].str.lower().str.strip()
grammys_df['artist_norm'] = grammys_df['artist'].str.lower().str.strip()

merged_df['artists_list'] = merged_df['artists_norm'].str.split(';')
exploded_spotify = merged_df.explode('artists_list')
exploded_spotify['artists_list'] = exploded_spotify['artists_list'].str.strip()

exact_match = pd.merge(
    exploded_spotify,
    grammys_df,
    left_on=['track_name_norm', 'artists_list'],
    right_on=['nominee_norm', 'artist_norm'],
    how='left',
    suffixes=('', '_grammy')
)

exact_match = exact_match.drop_duplicates(subset=['track_id'], keep='first')


no_match_mask = exact_match['id'].isna()

if no_match_mask.any():
    tracks_without_match = merged_df[merged_df['track_id'].isin(exact_match[no_match_mask]['track_id'])]
    
    song_match = pd.merge(
        tracks_without_match,
        grammys_df,
        left_on='track_name_norm',
        right_on='nominee_norm',
        how='left',
        suffixes=('', '_grammy_song')
    )
    
    song_match = song_match[song_match['id'].notna()]
    song_match = song_match.drop_duplicates(subset=['track_id'], keep='first')
    
    final_df = pd.concat([
        exact_match[~no_match_mask],
        song_match
    ], ignore_index=True)
else:
    final_df = exact_match


all_matched_ids = final_df['track_id'].unique()
remaining_tracks = merged_df[~merged_df['track_id'].isin(all_matched_ids)]
final_df = pd.concat([final_df, remaining_tracks], ignore_index=True)

final_df = final_df.drop(columns=[
    'track_name_norm', 'artists_norm', 'artists_list',
    'nominee_norm', 'artist_norm',
    'nominee', 'artist'
], errors='ignore')


final_df = final_df.rename(columns={
    'id': 'grammy_id',
    'year': 'grammy_year',
    'category': 'grammy_category'
})

final_df = final_df.drop_duplicates(subset=['track_id'])

final_df.to_csv(output_path, index=False)
print(f"Merge completado. Archivo guardado como {output_path}")
print(f"Registros originales: {len(spotify_df)}")
print(f"Registros finales: {len(final_df)}")
print(f"Tracks con match en Grammys: {final_df['grammy_id'].notna().sum()}")

Merge completado. Archivo guardado como data/merge_dataset.csv
Registros originales: 89740
Registros finales: 89740
Tracks con match en Grammys: 1737


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns    

final_csv_path = 'data/merge_dataset.csv'
df = None 

try:
    df = pd.read_csv(final_csv_path)
    print(f"Dataset '{final_csv_path}' cargado exitosamente.")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{final_csv_path}'.")
    print("Asegúrate de que el script de merge se ejecutó correctamente y generó el archivo.")
except Exception as e:
    print(f"Error al cargar el archivo CSV: {e}")

if df is None:
    raise SystemExit("Deteniendo ejecución: El DataFrame no pudo ser cargado.")

Dataset 'data/merge_dataset.csv' cargado exitosamente.


/tmp/ipykernel_24561/3195924198.py:16: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(final_csv_path)


In [ ]:
print("\n--- Inspección Básica ---")

print(f"\nDimensiones del DataFrame (Filas, Columnas): {df.shape}")

print("\nPrimeras 5 filas:")
display(df.head(20)) 

print("\nInformación General (dtypes, non-null counts):")
df.info()

print("\nColumnas presentes:")
print(df.columns.tolist())


--- Inspección Básica ---

Dimensiones del DataFrame (Filas, Columnas): (89740, 22)

Primeras 5 filas:


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,loudness,...,valence,tempo,track_genre,genre_category,artist_id,artist_followers,artist_popularity,grammy_id,grammy_year,grammy_category
0,5svyefkggqeycgnqr4kslq,joy williams,front porch,front porch,64,231813,False,0.567,0.224,-10.024,...,0.200,124.440,acoustic,folk/acoustic,4tcxgddpm10enslncvniya,172508,54,267.0,2019.0,best folk album
1,18k8ils52uxrwobykduzfg,brandi carlile,"by the way, i forgive you",the joke,60,279373,False,0.236,0.578,-5.477,...,0.227,75.367,acoustic,folk/acoustic,2sg4ztolvjkg1psooyf5ej,883852,67,480.0,2018.0,best american roots performance
2,4fvkqlgqlsj7gilicaxtxl,julieta venegas,éxitos del pop en español de los años 2000,limón y sal,0,205213,False,0.625,0.653,-5.967,...,0.605,110.972,alt-rock,rock,2qwiscpfdnxms6zemiuvgm,2923638,76,1600.0,2006.0,best latin pop album
3,7omo1b9cb4zpypleve32oy,r.e.m.,90's music grandi successi,losing my religion,0,268426,False,0.666,0.854,-5.051,...,0.803,125.637,alt-rock,rock,4kwtalx2rvbpseogmemrog,5064343,75,2994.0,1991.0,best pop performance by a duo or group with vocal
4,31aoj9sfz2gm0o3hmarrbx,r.e.m.,out of time (25th anniversary edition),losing my religion,84,268426,False,0.666,0.855,-5.051,...,0.803,125.639,alt-rock,rock,4kwtalx2rvbpseogmemrog,5064343,75,2994.0,1991.0,best pop performance by a duo or group with vocal
5,1zksmkyorose4pw41njrek,no doubt;bounty killer,oo's music grandi successi,hey baby,0,207040,False,0.705,0.871,-3.557,...,0.749,93.625,alt-rock,rock,0cqbju1aazvbemtuljwllf,3016218,70,1978.0,2002.0,best pop performance by a duo or group with vocal
6,2vgdtotslhkco6lvffdei2,soundgarden,rock & halloween,black hole sun,0,318586,False,0.351,0.827,-5.351,...,0.145,105.463,alt-rock,rock,5xuf6j4upbrxzpg6ai4mrk,4796088,70,2754.0,1994.0,best hard rock performance
7,6jn0cjft0zdryxzqckrvwf,soundgarden,hits pop rock 90's,spoonman,0,246920,False,0.258,0.904,-5.625,...,0.832,186.054,alt-rock,rock,5xuf6j4upbrxzpg6ai4mrk,4796088,70,2755.0,1994.0,best metal performance
8,0bavcb3mn8qggudei2yqcb,stone temple pilots,timeless rock hits,plush,1,310346,False,0.416,0.954,-3.157,...,0.498,144.724,alt-rock,rock,2uazatjfzqbf0nho2awk4z,3786771,71,2839.0,1993.0,best hard rock performance with vocal
9,7kpwpv8wlkkanrd4lloqt9,soundgarden,hits pop rock 90's,black hole sun,1,318586,False,0.351,0.827,-5.351,...,0.145,105.463,alt-rock,rock,5xuf6j4upbrxzpg6ai4mrk,4796088,70,2754.0,1994.0,best hard rock performance



Información General (dtypes, non-null counts):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89740 entries, 0 to 89739
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   track_id           89740 non-null  object 
 1   artists            89740 non-null  object 
 2   album_name         89740 non-null  object 
 3   track_name         89740 non-null  object 
 4   popularity         89740 non-null  int64  
 5   duration_ms        89740 non-null  int64  
 6   explicit           89740 non-null  bool   
 7   danceability       89740 non-null  float64
 8   energy             89740 non-null  float64
 9   loudness           89740 non-null  float64
 10  acousticness       89740 non-null  float64
 11  instrumentalness   89740 non-null  float64
 12  valence            89740 non-null  float64
 13  tempo              89740 non-null  float64
 14  track_genre        89740 non-null  object 
 15  genre_category     897

In [ ]:
print("\n--- Conteo de Valores Faltantes por Columna ---")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])


--- Conteo de Valores Faltantes por Columna ---
id          88003
year        88003
category    88003
nominee     88003
artist      88003
dtype: int64


In [23]:
print("\n--- Estadísticas Descriptivas (Columnas Numéricas) ---")
print(df.describe())


--- Estadísticas Descriptivas (Columnas Numéricas) ---
         popularity   duration_ms  danceability        energy      loudness  \
count  90081.000000  9.008100e+04   90081.00000  90081.000000  90081.000000   
mean      33.223421  2.291235e+05       0.56220      0.634293     -8.498493   
std       20.606697  1.127964e+05       0.17666      0.256609      5.220365   
min        0.000000  8.586000e+03       0.00000      0.000000    -49.531000   
25%       19.000000  1.730610e+05       0.45000      0.457000    -10.324000   
50%       33.000000  2.133060e+05       0.57600      0.676000     -7.185000   
75%       49.000000  2.642660e+05       0.69200      0.853000     -5.108000   
max      100.000000  5.237295e+06       0.98500      1.000000      4.532000   

       acousticness  instrumentalness       valence         tempo  \
count  90081.000000      90081.000000  90081.000000  90081.000000   
mean       0.328200          0.173415      0.469251    122.062302   
std        0.338315      

In [24]:
print("\n--- Conteo de Valores Únicos (Categorías de Género - Top 10) ---")
print(df['genre_category'].value_counts().head(10))


--- Conteo de Valores Únicos (Categorías de Género - Top 10) ---
electronic                17851
other                     11494
world/regional             9791
rock                       8335
latin                      7228
pop                        7106
classical/instrumental     5492
kids/comedy                4868
metal                      4834
folk/acoustic              3384
Name: genre_category, dtype: int64


In [5]:
from dags.tasks.store import store_merged_data

In [6]:
store_merged_data('merged_data', df)

11/04/2025 10:23:00 PM Starting Google Drive authentication process.
11/04/2025 10:23:00 PM Client secrets path: drive_config/client_secrets.json
11/04/2025 10:23:00 PM Settings file path: env/settings.yaml
11/04/2025 10:23:00 PM Credentials file path: drive_config/saved_credentials.json
11/04/2025 10:23:00 PM Folder ID: 1-3BNlMIxSYC2vlwVjta624VIL2dzFPOf
11/04/2025 10:23:00 PM Saved credentials not found, performing web authentication.


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=767869104842-tnsv4h2v7c7u2kvekpq5iq6rd99m082e.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8090%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=online&response_type=code



11/04/2025 10:29:41 PM Received token response with no refresh_token. Consider reauthenticating with prompt='consent'.
11/04/2025 10:29:41 PM Successfully retrieved access token
11/04/2025 10:29:41 PM Local webserver authentication completed and credentials saved successfully.
11/04/2025 10:29:41 PM Google Drive authentication completed successfully.
11/04/2025 10:29:41 PM Storing merged_data on Google Drive.
11/04/2025 10:29:41 PM DataFrame has 89740 rows and 23 columns.


Authentication successful.


11/04/2025 10:30:02 PM File merged_data uploaded successfully.
